In [1]:
import notebook_importer

import numpy as np
import pickle
import random
import zlib

In [2]:
BASE = 10
KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

# Q field
Q = 6497992661811505123 # < 64 bits
Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

# P field
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits
P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

# TODO: this is a tmp solution. Remove asap!
socket_party = None
spdz_socket = None

In [3]:
def encode(rational, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = int(rational * BASE**precision_fractional)
    field_element = upscaled % field
    return field_element

def decode(field_element, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = field_element.value if field_element.value <= field/2 else field_element.value - field
    rational = upscaled / BASE**precision_fractional
    return rational

def share(secret, field=Q):
    first  = random.randrange(field)
    second = (secret - first) % field
    return [first, second]

def reconstruct(shares, field=Q):
    return sum(shares) % field

In [4]:
add_shares = np.vectorize(lambda a, b, q=Q: a.combine_with(b))

# only add the value to one of the shares
add_public_vec = np.vectorize(lambda a, b, q=Q: add_public(a, b, q))
def add_public(x, k, field=Q):
    if x.party == 0:
        return x + k
    return x

def sub_public(x, k, field=Q):
    if x.party == 0:
        return x - k
    return x

In [5]:
def generate_multiplication_triple(field=Q):
    a = random.randrange(field)
    b = random.randrange(field)
    c = (a * b) % field

    return a, b, c

In [6]:
class PrivateValue(object):
    
    def __init__(self, party, share=None, field=Q, is_shared=False):
        self.value = share
        self.field = field
        self.party = party
        self.is_shared = is_shared
    
    def secure(secret, field=Q):
        alice, bob = share(encode(secret))
        return PrivateValue(0, alice, field, True), PrivateValue(1, bob, field, True)

    def share(self):
        alice, bob = share(self.value)
        return PrivateValue(0, alice, self.field, True), PrivateValue(1, bob, self.field, True)

    def combine_with(self, share):
        if isinstance(share, PrivateValue):
            if share.field != self.field:
                raise ValueError("Numbers are not in the same field!")
            return PrivateValue(self.party, (self.value + share.value) % self.field, self.field)
        else:
            raise ValueError

    def unwrap(self):
        return self.value
    
    def __repr__(self):
        return "PrivateValue(%s)" % self.value

    def __mod__(self, mod):
        return PrivateValue(self.party, self.value % mod, self.field)
    
    def __add__(x, y):
        if isinstance(y, PrivateValue):
            if y.field != x.field:
                raise ValueError("Numbers are not in the same field!")
            return PrivateValue(x.party, (x.value + y.value) % x.field, x.field, x.is_shared or y.is_shared)
        else:
            raise ValueError
    
    def __sub__(x, y):
        #print("private_sub")
        if isinstance(y, PrivateValue):            
            if y.field != x.field:
                raise ValueError("Numbers are not in the same field!")
            return PrivateValue(x.party, (x.value - y.value) % x.field, x.field, x.is_shared or y.is_shared)  
        else:
            raise ValueError
    
    def __mul__(x, y):
        if isinstance(y, PrivateValue):
            if y.field != x.field:
                raise ValueError("Numbers are not in the same field!")
            if x.is_shared and y.is_shared:
                return mul_communication(x, y, socket_party, spdz_socket, field=x.field)
            return PrivateValue(x.party, (x.value * y.value) % x.field, x.field, x.is_shared or y.is_shared)  
        else:
            raise ValueError

In [7]:
wrap = np.vectorize(lambda x, q=Q: PrivateValue(socket_party, x, q))
unwrap = np.vectorize(lambda x: x.unwrap())

share_vec = np.vectorize(lambda x: x.share())

In [8]:
def generate_matmul_triple(m, k, n, field=Q):
    r = wrap(np.random.randint(field, size=(m, k)))
    s = wrap(np.random.randint(field, size=(k, n)))
    t = np.dot(r, s)
    return share_vec(r), share_vec(s), share_vec(t)

In [9]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    if x.party == 0:
        return PrivateValue(x.party, x.value // (BASE**amount), field, x.is_shared)
    return PrivateValue(x.party, field - ((field - x.value) // BASE**amount), field, x.is_shared) 

In [10]:
encode_vec = np.vectorize(lambda x: encode(x))
decode_vec = np.vectorize(lambda x: decode(x))

In [11]:
truncate_vec = np.vectorize(lambda x: truncate(x))

In [12]:
secure = np.vectorize(lambda x: PrivateValue.secure(x))

In [13]:
# TODO: should we put a check if the object is privatevalue?
def send_share(value, socket, flags=0, protocol=-1):
    """pickle an object, and zip the pickle before sending it"""
    p = pickle.dumps(value, protocol)
    z = zlib.compress(p)
    return socket.send(z, flags=flags)

def receive_share(socket, flags=0, protocol=-1):
    """inverse of send_zipped_pickle"""
    z = socket.recv(flags)
    p = zlib.decompress(z)
    return pickle.loads(p)

def swap_shares(share, party, socket):
    if (party == 0):
        send_share(share, socket)
        share_other = receive_share(socket)
    elif (party == 1):
        share_other = receive_share(socket)
        send_share(share, socket)
    return share_other

In [14]:
def combine_shares(share, party, socket):
    share_other = swap_shares(share, party, socket)
    return decode_vec(share + share_other)

In [15]:
def generate_multiplication_triple_communication(party, socket):
    # TODO: a third party should generate these triples
    if (party == 0):
        a, b, c = generate_multiplication_triple()

        a_alice, a_bob = PrivateValue(socket_party, a).share()
        b_alice, b_bob = PrivateValue(socket_party, b).share()
        c_alice, c_bob = PrivateValue(socket_party, c).share()

        triple_alice = [a_alice, b_alice, c_alice]
        triple_bob = [a_bob, b_bob, c_bob]
        response = swap_shares(triple_bob, party, socket)
        return triple_alice
    elif (party == 1):
        triple_bob = swap_shares(np.array("OK"), party, socket)
        return triple_bob

In [16]:
def mul_communication(x, y, party, socket, field=Q):
    # TODO: compute triples offline
    triple = generate_multiplication_triple_communication(party, socket)
    a, b, c = triple

    # local masking
    d = x - a
    e = y - b
    
    # communication: the players simultanously send their shares to the other
    d_other = swap_shares(d, party, socket)
    e_other = swap_shares(e, party, socket)

    delta = d.combine_with(d_other)
    epsilon = e.combine_with(e_other)

    r = delta * epsilon 

    s = a * epsilon
    t = b * delta

    share = s + t + c

    # imitate public_add
    share = add_public(share, r)
    share = truncate(share)
    return share

In [17]:
def generate_matmul_triple_communication(m, k, n, party, socket):
    # TODO: a third party should generate these triples
    
    if (party == 0):
        r, s, t = generate_matmul_triple(m, k, n)
        triple_alice = [r[0],s[0],t[0]]
        triple_bob = [r[1],s[1],t[1]]
        response = swap_shares(triple_bob, party, socket)
        return triple_alice
    
    elif (party == 1):
        triple_bob = swap_shares(np.array("OK"), party, socket)
        return triple_bob

In [18]:
def matmul_communication(x, y, party, socket, field=Q):
    
    # TODO: compute triples offline

    x_height = x.shape[0]
    x_width = x.shape[1]
    
    y_height = y.shape[0]
    y_width = y.shape[1]
    
    assert x_width == y_height
    
    r, s, t = generate_matmul_triple_communication(x_height, x_width, y_width, party, socket)

    rho_local = x - r
    sigma_local = y - s
    
    # Communication
    rho_other = swap_shares(rho_local, party, socket)
    sigma_other = swap_shares(sigma_local, party, socket)
    
    # They both add up the shares locally
    rho = add_shares(rho_local, rho_other)
    sigma = add_shares(sigma_local, sigma_other)

    r_sigma = np.dot(r, sigma)    
    rho_s = np.dot(rho, s)

    share =  r_sigma + rho_s + t 
    
    rs = np.dot(rho, sigma)

    share = add_public_vec(share, rs)
    share = truncate_vec(share)   
    
    return share 

In [19]:
def generate_statistical_mask():
    return random.randrange(2*BOUND * 10**KAPPA)

def generate_zero_triple(field):
    return share(0, field)


# TODO: above public add seems to add a new dimension that causes some problems

# below functions are used for public additions and subtractions. 
# they are applied to only one of the shares
new_add = np.vectorize(lambda x,y,q: (x + y) % q)
new_sub = np.vectorize(lambda x,y,q: (x - y) % q)

def convert(party, socket, x, from_field, to_field):
    if (party == 0):
        alice_zero, bob_zero = generate_zero_triple(to_field)
        swap_shares(np.array([bob_zero]), party, socket)
        # local mapping to positive representation
        alice_x = new_add(x, BOUND, from_field)
        # local masking and conversion by player 0
        r = generate_statistical_mask()
        alice = np.ones(alice_x.shape) * ((alice_zero - r) % to_field)
        # exchange of masked share: one round of communication
        e = (x + r) % from_field
        swap_shares(np.array([e]), party, socket)
        # local mapping back from positive representation
        alice = new_sub(alice, BOUND, to_field)
        return alice
    elif (party == 1):
        bob_zero = swap_shares(np.array("OK"), party, socket)
        bob_e = swap_shares(np.array("OK"), party, socket)[0]
        # Warning: xr leaks information about x. so, it is better to send this share back to the data owner.
        xr = (bob_e + x) % from_field
        bob = (bob_zero + xr) % to_field
        return bob

def upshare(party, socket, x):
    return convert(party, socket, x, Q, P)

def downshare(party, socket, x):
    return convert(party, socket, x, P, Q)

In [20]:
def generate_powering_triple_communication(party, socket, exponent=Q_MAXDEGREE, field=Q):
    if (party == 0):
        a = random.randrange(field)
        alice, bob = map(np.array, zip(*[ share(pow(a, e) % field, field) for e in range(1, exponent+1) ]))
        swap_shares(bob, party, socket)
        return alice
    elif (party == 1):
        bob = swap_shares(np.array("OK"), party, socket)
        return bob

In [21]:
def binomial(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke.
    See http://stackoverflow.com/questions/3025162/statistics-combinations-in-python
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0



class SigmoidInterpolated10:
    
    #TODO: share Coeffs    
    def __init__(self, party, socket, field=Q):
        self.party = party
        self.socket = socket

        self.ONE = self.swap_one(field)
        
        # self.sigmoid_coeffs = [x for x in map(partial(encode, field=P), [0.5, 0.2159198015, 0, -0.0082176259, 0, 0.0001825597, 0, -0.0000018848, 0, 0.0000000072])]
        self.sigmoid_deriv = np.vectorize(lambda x:(self.ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)
    
    def sigmoid(self, x):
        # return self.approximate(x)
        return self.approximate_with_small_field(x)

    def approximate_with_small_field(self, x):
        field = Q

        # currently we are using 7 numbers for precision
        W0  = PrivateValue(self.party, encode(0.5))
        W1  = PrivateValue(self.party, encode(0.2159198))  #015))
        W3  = PrivateValue(self.party, encode(-0.0082176)) #259))
        W5  = PrivateValue(self.party, encode(0.0001825))  #597))
        W7  = PrivateValue(self.party, encode(-0.0000018)) #848))
        W9  = PrivateValue(self.party, encode(0.0000000))  #072))

        x2 = np.multiply(x, x)
        x3 = np.multiply(x2, x) 
        x5 = np.multiply(x2, x3)                    
        x7 = np.multiply(x2, x5)
        x9 = np.multiply(x2, x7) 

        # TODO: need a better solution to truncate after mul_public. i.e. x9, 
        result = np.multiply(x9, W9) + truncate_vec(np.multiply(x7, W7)) \
                + truncate_vec(np.multiply(x5, W5)) + truncate_vec(np.multiply(x3, W3)) \
                + truncate_vec(np.multiply(x, W1))

        result = add_public_vec(result, W0)
        return result

    def swap_one(self, field):
        if self.party == 0:
            alice, bob = PrivateValue.secure(1)
            swap_shares(np.array([bob]), self.party, self.socket)
            return alice
        elif self.party == 1:
            bob = swap_shares(np.array("OK"), self.party, self.socket)[0]
            return bob

    # TODO: fix below implementation
    def approximate(self, x):
        field = P

        triple = generate_powering_triple_communication(self.party, self.socket, 9, field)

        ONE = self.swap_one(field)

        up_x = upshare(self.party, self.socket, x)

        # local masking
        a = triple[0]
        v = new_sub(x, a, field)
        # communication: the players simultanously send their share to the other
        v_share = swap_shares(v, self.party, self.socket)
        epsilon = new_add(v, v_share, field) #reconstruct(v)
        # local combination to compute all powers
        x_powers = []
        for exponent in range(1, len(triple)+1):
            # prepare all term values
            a_powers = [ONE] + triple[:exponent]
            e_powers = [ pow(epsilon, e) % field for e in range(exponent+1) ]
            coeffs   = [ binomial(exponent, k) for k in range(exponent+1) ]
            # compute and sum terms
            terms = [ (a * e * c * s) % field for a,e,c,s in zip(a_powers,reversed(e_powers),coeffs,self.sigmoid_coeffs) ]
            x_powers.append(terms)

        powers = []
        for i, x_pow in enumerate(x_powers):
            combined = (swap_shares(np.asarray(x_pow, dtype=object), self.party, self.socket) + x_pow) % field
            powers.append((ONE + combined) % field)

        terms = [ ((xe * ce) % field) for xe,ce in zip(powers, coeffs) ]

        combined_terms = []
        for i, term in enumerate(terms):
            combined_terms.append((swap_shares(np.asarray(term, dtype=object), self.party, self.socket) + term) % field)


        return downshare(self.party, self.socket, terms)

In [22]:
class TwoLayerNetwork:

    def __init__(self, sigmoid, party, socket):
        self.sigmoid = sigmoid
        self.party = party
        self.socket = socket
    
    def train(self, X, y, syn0, iterations=100, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = syn0
        
        # training
        for i in range(iterations):
            layer0 = X

            # TODO: sigmoid hack combines shares: use powering triple to make it secure
            layer1 = matmul_communication(layer0, self.synapse0, self.party, self.socket)            
            layer1 = self.sigmoid.evaluate(layer1)
            
            # back propagation 
            # TODO: truncating makes the matrix all zeros!
            layer1_error = y - layer1
            layer1_delta = np.multiply(layer1_error, self.sigmoid.derive(layer1))
            #layer1_delta = truncate_vec(np.multiply(layer1_error, self.sigmoid.derive(layer1)))

            new_synapse0 = matmul_communication(layer0.T, layer1_delta, self.party, self.socket)
            
            self.synapse0 = np.add(self.synapse0, new_synapse0)
                        
    def print_weights(self):
        weights = combine_shares(self.synapse0, self.party, self.socket)
        print("Layer 0 weights: \n%s" % weights)
        return weights

    def predict(self, X):
        layer0 = X
        layer1 = matmul_communication(layer0, self.synapse0, self.party, self.socket)
        layer1 = self.sigmoid.evaluate(layer1)
        return layer1